In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:
nc = 5

def get_rand_rects_mask(nc):
    x1, y1 = 0.5, 0.5
    x2, y2 = 0.5, 0.5
    while 0<x1<1 or 0<y1<1 or 0<x2<1 or 0<y2<1:
        cx, cy = random.random(), random.random()
        rad = (1.714/nc + random.random()) * nc
        ang = 6.29 * random.random()
        x1, y1 = cx + (rad / 2.0) * math.cos(ang), cy + (rad / 2.0) * math.sin(ang)
        x2, y2 = cx + (rad / 2.0) * math.cos(ang+3.1415), cy + (rad / 2.0) * math.sin(ang+3.1415)
    return x1, y1, x2, y2
from enum import Enum

class Crop(Enum):
    NONE = 0
    HORIZONTAL_SHIFT = 1
    VERTICAL_SHIFT = 2
    CIRCLE_SHIFT = 3
    CIRCLE_NOSHIFT = 4
    HORIZONTAL_STRETCH = 5
    VERTICAL_STRETCH = 6
    
crops = [Crop.NONE, Crop.HORIZONTAL_SHIFT, Crop.VERTICAL_SHIFT, Crop.CIRCLE_SHIFT, Crop.CIRCLE_NOSHIFT, Crop.HORIZONTAL_STRETCH, Crop.VERTICAL_STRETCH]
crop = crops[int(random.random()*len(crops))]
print(crop)
from util import *
ch = get_channels_from_filename("H1-102+D3-100+F6-17+G2-36+F1-40+D6-32+D3-27+F6-42+F5-44+H6-72+D2-13+D1-186+E1-100+D1-162+E1-115+G1-36+E1-100+E2-62+D1-67+D1-96+F3-187+H1-116+H3-7+D3-92+H6-26+D1-144_n20_o04_r1.50_cr4_b0.25_1.15,1.25,1.24")
print(ch)
print(len(ch))

Crop.NONE
Number of layers 59
Total number of feature channels: 7548
[{'layer': 'H1', 'channel': 102}, {'layer': 'D3', 'channel': 100}, {'layer': 'F6', 'channel': 17}, {'layer': 'G2', 'channel': 36}, {'layer': 'F1', 'channel': 40}, {'layer': 'D6', 'channel': 32}, {'layer': 'D3', 'channel': 27}, {'layer': 'F6', 'channel': 42}, {'layer': 'F5', 'channel': 44}, {'layer': 'H6', 'channel': 72}, {'layer': 'D2', 'channel': 13}, {'layer': 'D1', 'channel': 186}, {'layer': 'E1', 'channel': 100}, {'layer': 'D1', 'channel': 162}, {'layer': 'E1', 'channel': 115}, {'layer': 'G1', 'channel': 36}, {'layer': 'E1', 'channel': 100}, {'layer': 'E2', 'channel': 62}, {'layer': 'D1', 'channel': 67}, {'layer': 'D1', 'channel': 96}, {'layer': 'F3', 'channel': 187}, {'layer': 'H1', 'channel': 116}, {'layer': 'H3', 'channel': 7}, {'layer': 'D3', 'channel': 92}, {'layer': 'H6', 'channel': 26}, {'layer': 'D1', 'channel': 144}]
26


In [1]:
## %matplotlib inline
import IPython.display
from PIL import Image, ImageStat, ImageEnhance
import math
import random
import os
from os import listdir
from os.path import isfile, isdir, join
import numpy as np
import scipy.misc
from tqdm import tqdm


def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 1)*255)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
    
def visstd(a, s=0.1):
    '''Normalize the image range for visualization'''
    return (a-a.mean())/max(a.std(), 1e-4)*s + 0.5

def histogram_equalization(x):
    hist, bins = np.histogram(x.flatten(), 255, [0, 256])
    cdf = hist.cumsum()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255.0/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    x2 = cdf[x.astype('uint8')]
    return x2, cdf

def get_histogram(pixels, bright=True):
    hist = np.zeros((256,1 if bright else 3))
    for p in pixels:
        if bright:
            avg = int((p[0]+p[1]+p[2])/3.0)
            hist[avg,0] += 1
        else:
            hist[p[0],0] += 1
            hist[p[1],1] += 1
            hist[p[2],2] += 1
    return np.array(hist)

def match_histogram2(img1, hist):
    colors = img1.getdata()
    red, green, blue = [c[0] for c in colors], [c[1] for c in colors], [c[2] for c in colors]
    sr = sorted(range(len(red)), key=lambda k: red[k])
    sg = sorted(range(len(green)), key=lambda k: green[k])
    sb = sorted(range(len(blue)), key=lambda k: blue[k])
    hr, hg, hb = [[hist[i][c] for i in range(256)] for c in range(3)]
    fr, fg, fb = 0,0,0
    for c in range(len(hr)):
        nfr, nfg, nfb = int(hr[c]), int(hg[c]), int(hb[c])
        idxr = [sr[k] for k in range(fr,fr+nfr)]
        idxg = [sg[k] for k in range(fg,fg+nfg)]
        idxb = [sb[k] for k in range(fb,fb+nfb)]
        for ir in idxr:
            red[ir] = c
        for ig in idxg:
            green[ig] = c
        for ib in idxb:
            blue[ib] = c
        fr, fg, fb = fr+nfr, fg+nfg, fb+nfb
    adjusted_colors = zip(red, green, blue)
    img_adjusted = Image.new(img1.mode, img1.size)
    img_adjusted.putdata(adjusted_colors)
    return img_adjusted

def match_histogram(img1, hist):
    pixels = list(img1.getdata())
    red, green, blue = np.array([c[0] for c in pixels]), np.array([c[1] for c in pixels]), np.array([c[2] for c in pixels])
    sr = sorted(range(len(red)), key=lambda k: red[k])
    sg = sorted(range(len(green)), key=lambda k: green[k])
    sb = sorted(range(len(blue)), key=lambda k: blue[k])
    num_pixel_mult = (3 * len(pixels)) / np.sum(hist)
    hr, hg, hb = [[int(num_pixel_mult * hist[i][c]) for i in range(256)] for c in range(3)]
    fr, fg, fb = 0, 0, 0
    for c in range(len(hr)):
        nfr, nfg, nfb = int(hr[c]), int(hg[c]), int(hb[c])
        red[np.array([sr[k] for k in xrange(fr,fr+nfr)]).astype('int')] = c
        green[np.array([sg[k] for k in xrange(fg,fg+nfg)]).astype('int')] = c
        blue[np.array([sb[k] for k in xrange(fb,fb+nfb)]).astype('int')] = c
        fr, fg, fb = fr+nfr, fg+nfg, fb+nfb
    adjusted_pixels = zip(red, green, blue)
    img_adjusted = Image.new(img1.mode, img1.size)
    img_adjusted.putdata(adjusted_pixels)
    return img_adjusted

def adjust_color_range(img, hist, amt, border):
    cdf = hist.cumsum() / np.sum(hist)
    i1, i2 = min([i for i in range(256) if cdf[i]>border]), max([i for i in range(256) if cdf[i]<1.0-border])
    j1, j2 = int((1.0-amt)*i1), i2 + amt*(255-i2)
    img2 = np.clip(j1 + (j2-j1)*(img - i1)/(i2-i1), 0.0, 255.0)
    return img2

def get_average_histogram(frames_path):
    numframes = len([f for f in listdir(frames_path) if isfile(join(frames_path, f)) and f[-4:]=='.png'])
    img = Image.open('%s/frame0001.png'%(frames_path))
    histogram = get_histogram(list(img.getdata()))
    for t in tqdm(range(1,numframes,8)):
        img = Image.open('%s/frame%04d.png'%(frames_path, t+1))
        histogram += get_histogram(list(img.getdata()))
    histogram /= (1+len(range(1,numframes,8)))
    return histogram

def generate_video2(frames_path, output_path):
    numframes = len([f for f in listdir(frames_path) if isfile(join(frames_path, f))])
    print('loading %d images from %s' % (numframes, frames_path))
    images = [ Image.open('%s/frame%04d.png'%(frames_path, t+1)) for t in range(numframes) ]
    brightness = [ ImageStat.Stat(img.convert('L')).mean[0] for img in images ]
    avg_brightness = np.mean(brightness)
    os.system('mkdir %s/temp' % frames_path)
    for i, img in tqdm(enumerate(images)):
        mult = avg_brightness / brightness[i]
        source = img.split()
        r = source[0].point(lambda i: i*mult)
        g = source[1].point(lambda i: i*mult)
        b = source[2].point(lambda i: i*mult)
        img2 = Image.merge(img.mode, (r, g, b))
        img2 = ImageEnhance.Contrast(img2).enhance(1.2)
        img2 = ImageEnhance.Sharpness(img2).enhance(1.25)
        scipy.misc.imsave("%s/temp/frame%04d.png" % (frames_path, i+1), img2)
    w, h  = images[0].size
    cmd = 'ffmpeg -i %s/temp/frame%%04d.png -c:v libx264 -pix_fmt yuv420p -vf scale=%d:%d %s'%(frames_path, w-(w%2), h-(h%2), output_path)
    print('creating movie at %s' % output_path)
    os.system('rm %s' % output_path)
    os.system(cmd)
    os.system('rm -rf %s/temp' % frames_path)
    print('done!')

def generate_video(frames_path, output_path, sat, con, sharp, bitrate=None, cumulative=False):
    numframes = len([f for f in listdir(frames_path) if isfile(join(frames_path, f)) and f[-4:]=='.png'])
    if numframes == 0:
        print("no frames found in %s"%frames_path)
        return
    print('creating %d-frame movie: %s -> %s'%(numframes, frames_path, output_path))
#     avg_hist = get_average_histogram(frames_path)
    os.system('mkdir %s/temp' % frames_path)
    for i in tqdm(range(numframes)):
        edited_filepath = "%s/temp/frame%04d.png" % (frames_path, i+1)
        if not os.path.isfile(edited_filepath) or not cumulative:
            img = Image.open('%s/frame%04d.png'%(frames_path, i+1))
    #         img = match_histogram(img, avg_hist)
            img = ImageEnhance.Color(img).enhance(sat)
            img = ImageEnhance.Contrast(img).enhance(con)
            img = ImageEnhance.Sharpness(img).enhance(sharp)
            scipy.misc.imsave(edited_filepath, img)
    w, h  = img.size
    wx, wy = w-(w%2), h-(h%2)
    #wx, wy = 600, 360
    if bitrate==None:
        cmd = 'ffmpeg -i %s/temp/frame%%04d.png -c:v libx264 -pix_fmt yuv420p -vf scale=%d:%d %s'%(frames_path, wx, wy, output_path)
    else:
        cmd = 'ffmpeg -i %s/temp/frame%%04d.png -c:v libx264 -pix_fmt yuv420p -vf scale=%d:%d -b %d %s'%(frames_path, wx, wy, bitrate, output_path)
    #cmd = 'ffmpeg -i %s/temp/frame%%04d.png -c:v libx264 -preset ultrafast -crf 0 -vf scale=%d:%d %s'%(frames_path, w-(w%2), h-(h%2), output_path)
    os.system('rm %s' % output_path)
    os.system(cmd)
    if not cumulative:
        os.system('rm -rf %s/temp' % frames_path)
    print('done!')


def gen_video(frames_path, sat, con, sharp, bitrate=None, cumulative=False):
    output_path = '%s_%0.2f,%0.2f,%0.2f.mp4'%(frames_path, sat, con, sharp)
    generate_video(frames_path, output_path, sat, con, sharp, bitrate, cumulative)

def gen_video_dir(frames_path, sat, con, sharp, overwrite=True, bitrate=None, cumulative=False):
    dirs = [f for f in listdir(frames_path) if isdir(join(frames_path, f))]
    for d in dirs:
        output_path = '%s/%s_%0.2f,%0.2f,%0.2f.mp4'%(frames_path, d, sat, con, sharp)
        file_exists = os.path.isfile(output_path)
        if overwrite or not file_exists:
            print("generate %s"%output_path)
            gen_video(join(frames_path, d), sat, con, sharp, bitrate, cumulative)


s,c,sh = 1.0,1.1,1.2

gen_video_dir("/home/gene/lapnorm/results/new9",s, c, sh, overwrite=True)


#gen_video("/home/gene/lapnorm/results/new2/C1-1_n10_o03_r1.32", s, c, sh, None, False)

#gen_video("/home/gene/lapnorm/results/new2/D1-152+D1-120+H4-13+G2-30+H3-103_n15_o02_r1.32", s, c, sh, None, False)
#gen_video("/home/gene/lapnorm/results/new3/C6-54+D2-50+C2-6+C6-5_n12_o02_r1.32", s, c, sh, None, False)



#gen_video("/home/gene/lapnorm/results/new3/C1-7+C6-30+B3-62+C6-5_n10_o02_r1.32", s, c, sh, None, False)
#gen_video("/home/gene/lapnorm/results/new2/F5-50+D3-100+F6-54+D1-162_n15_o03_r1.32", s, c, sh, None, False)

# faves = ['/home/gene/lapnorm/results/new2/C1-1_n10_o03_r1.32', 
#         '/home/gene/lapnorm/results/new2/H6-88_n10_o03_r1.32',
#         '/home/gene/lapnorm/results/new2/C6-5_n10_o03_r1.32',
#         '/home/gene/lapnorm/results/new2/H3-77_n10_o03_r1.32',
#         '/home/gene/lapnorm/results/new2/G2-36_n10_o03_r1.32',
#         '/home/gene/lapnorm/results/new2/H1-183_n10_o03_r1.32']
# for g in faves:
#     gen_video(g, s, c, sh, None, False)

  4%|▍         | 1/24 [00:00<00:03,  7.09it/s]

generate /home/gene/lapnorm/results/new9/B2-10+B3-62+B6-15+C3-179+C2-44_n08_o02_r1.39_1.00,1.10,1.20.mp4
creating 24-frame movie: /home/gene/lapnorm/results/new9/B2-10+B3-62+B6-15+C3-179+C2-44_n08_o02_r1.39 -> /home/gene/lapnorm/results/new9/B2-10+B3-62+B6-15+C3-179+C2-44_n08_o02_r1.39_1.00,1.10,1.20.mp4


  0%|          | 0/24 [00:00<?, ?it/s]

done!
generate /home/gene/lapnorm/results/new9/B6-15+C5-39+B2-10+B4-10+B3-62_n08_o02_r1.39_1.00,1.10,1.20.mp4
creating 24-frame movie: /home/gene/lapnorm/results/new9/B6-15+C5-39+B2-10+B4-10+B3-62_n08_o02_r1.39 -> /home/gene/lapnorm/results/new9/B6-15+C5-39+B2-10+B4-10+B3-62_n08_o02_r1.39_1.00,1.10,1.20.mp4


  4%|▍         | 1/24 [00:00<00:03,  5.80it/s]

...

done!
generate /home/gene/lapnorm/results/new9/C5-13+C3-11+B2-54+B3-57+B3-54_n08_o02_r1.39_1.00,1.10,1.20.mp4
creating 24-frame movie: /home/gene/lapnorm/results/new9/C5-13+C3-11+B2-54+B3-57+B3-54_n08_o02_r1.39 -> /home/gene/lapnorm/results/new9/C5-13+C3-11+B2-54+B3-57+B3-54_n08_o02_r1.39_1.00,1.10,1.20.mp4


  4%|▍         | 1/24 [00:00<00:03,  5.99it/s]

done!
generate /home/gene/lapnorm/results/new9/C6-38+B4-10+C3-1+C2-44+B1-31_n08_o02_r1.39_1.00,1.10,1.20.mp4
creating 24-frame movie: /home/gene/lapnorm/results/new9/C6-38+B4-10+C3-1+C2-44+B1-31_n08_o02_r1.39 -> /home/gene/lapnorm/results/new9/C6-38+B4-10+C3-1+C2-44+B1-31_n08_o02_r1.39_1.00,1.10,1.20.mp4


100%|██████████| 24/24 [00:04<00:00,  5.99it/s]


done!


In [17]:


from bookmarks import *
from PIL import Image, ImageDraw, ImageFont
import math



layer_lookup={'A1':'conv2d0_pre_relu', 'A2':'conv2d1_pre_relu', 'A3':'conv2d2_pre_relu', 'B1':'mixed3a_1x1_pre_relu', 'B2':'mixed3a_3x3_bottleneck_pre_relu', 'B3':'mixed3a_3x3_pre_relu', 'B4':'mixed3a_5x5_bottleneck_pre_relu', 'B5':'mixed3a_5x5_pre_relu', 'B6':'mixed3a_pool_reduce_pre_relu', 'C1':'mixed3b_1x1_pre_relu', 'C2':'mixed3b_3x3_bottleneck_pre_relu', 'C3':'mixed3b_3x3_pre_relu', 'C4':'mixed3b_5x5_bottleneck_pre_relu', 'C5':'mixed3b_5x5_pre_relu', 'C6':'mixed3b_pool_reduce_pre_relu', 'D1':'mixed4a_1x1_pre_relu', 'D2':'mixed4a_3x3_bottleneck_pre_relu', 'D3':'mixed4a_3x3_pre_relu', 'D4':'mixed4a_5x5_bottleneck_pre_relu', 'D5':'mixed4a_5x5_pre_relu', 'D6':'mixed4a_pool_reduce_pre_relu', 'E1':'mixed4b_1x1_pre_relu', 'E2':'mixed4b_3x3_bottleneck_pre_relu', 'E3':'mixed4b_3x3_pre_relu', 'E4':'mixed4b_5x5_bottleneck_pre_relu', 'E5':'mixed4b_5x5_pre_relu', 'E6':'mixed4b_pool_reduce_pre_relu', 'F1':'mixed4c_1x1_pre_relu', 'F2':'mixed4c_3x3_bottleneck_pre_relu', 'F3':'mixed4c_3x3_pre_relu', 'F4':'mixed4c_5x5_bottleneck_pre_relu', 'F5':'mixed4c_5x5_pre_relu', 'F6':'mixed4c_pool_reduce_pre_relu', 'G1':'mixed4d_1x1_pre_relu', 'G2':'mixed4d_3x3_bottleneck_pre_relu', 'G3':'mixed4d_3x3_pre_relu', 'G4':'mixed4d_5x5_bottleneck_pre_relu', 'G5':'mixed4d_5x5_pre_relu', 'G6':'mixed4d_pool_reduce_pre_relu', 'H1':'mixed4e_1x1_pre_relu', 'H2':'mixed4e_3x3_bottleneck_pre_relu', 'H3':'mixed4e_3x3_pre_relu', 'H4':'mixed4e_5x5_bottleneck_pre_relu', 'H5':'mixed4e_5x5_pre_relu', 'H6':'mixed4e_pool_reduce_pre_relu', 'J1':'mixed5a_1x1_pre_relu', 'J2':'mixed5a_3x3_bottleneck_pre_relu', 'J3':'mixed5a_3x3_pre_relu', 'J4':'mixed5a_5x5_bottleneck_pre_relu', 'J5':'mixed5a_5x5_pre_relu', 'J6':'mixed5a_pool_reduce_pre_relu', 'K1':'mixed5b_1x1_pre_relu', 'K2':'mixed5b_3x3_bottleneck_pre_relu', 'K3':'mixed5b_3x3_pre_relu', 'K4':'mixed5b_5x5_bottleneck_pre_relu', 'K5':'mixed5b_5x5_pre_relu', 'K6':'mixed5b_pool_reduce_pre_relu', 'L1':'head0_bottleneck_pre_relu', 'L2':'head1_bottleneck_pre_relu'}

layers_c1 = [   
    {"layer":'A1', "channels":[7,15,1,28,52,8]},
    {"layer":'A2', "channels":[12,3,38,25,27,18,29,14]},
    {"layer":'A3', "channels":[159,30,20,170,130,167,89,150,28,124,31,58,19,75,69]}
]

layers_m3 = [
    {"layer":'B1', "channels":[5,55,21,38,11,60,42,61,58,39,37,8,31,20,52,9]},
    {"layer":'B2', "channels":[3,6,42,57,54,26,44,40,10,19,0,30,36,45,38]},
    {"layer":'B3', "channels":[15,13,7,43,57,4,27,56,61,62,54,51,39,64,76,113]},
    {"layer":'B4', "channels":[10,4,2,7]},
    {"layer":'B5', "channels":[12,2]},
    {"layer":'B6', "channels":[13,14,15,6,7]},

    {"layer":'C1', "channels":[7,14,2,5,10,9,1,33,46,81,106,108,95,110,127]},
    {"layer":'C2', "channels":[13,6,10,0,8,44]},
    {"layer":'C3', "channels":[10,0,3,12,6,1,11,7,123,128,179,183,188]},
    {"layer":'C4', "channels":[10,1,11,14,15]},
    {"layer":'C5', "channels":[13,1,4,9,5,0,3,39,53,87,88]},
    {"layer":'C6', "channels":[5,15,6,9,4,8,3,2,19,30,35,38,54]}
]

layers_m4 = [
    {"layer":'D1', "channels":[36,63,46,144,178,138,35,1,43,41,96,30,51,97,121,127,149,54,101,120,88,4,3,170,158,81,68,131,55,110,162,137,86,118,85,21,189,191,155,14,172,133,20,57,2,67,38,130,186,112,164,176,11,27,117,103,163,32,87,152]},
    {"layer":'D2', "channels":[53,92,42,84,6,50,24,55,91,59,38,8,35,20,73,13]},
    {"layer":'D3', "channels":[68,4,47,21,32,1,82,85,48,27,65,92,33,100,111,118,125,130,138,139,147,174,185,201]},
    {"layer":'D4', "channels":[5]},
    {"layer":'D5', "channels":[15]},
    {"layer":'D6', "channels":[14,9,12,3,11,15,20,31,32,52]},

    {"layer":'E1', "channels":[58,23,42,125,6,100,62,31,115,68]},
    {"layer":'E2', "channels":[91,45,81,7,109,85,52,62,78,37]},
    {"layer":'E3', "channels":[95,78,59,36,82,17,50,15,127,130,150,172,188,189,196]},
    {"layer":'E4', "channels":[15,14,21]},
    {"layer":'E5', "channels":[3]},
    {"layer":'E6', "channels":[0,22,14,12,1,16,17,21,27,33,37,38,44,50,63]},

    {"layer":'F1', "channels":[13,22,30,31,40,49,78,83,84,94]},
    {"layer":'F2', "channels":[6,7,8,18,32,41,64,79,80,85,120,126]},
    {"layer":'F3', "channels":[48,75,33,124,10,46,95,107,26,109,147,164,187,230,245,254]},
    {"layer":'F4', "channels":[20,21]},
    {"layer":'F5', "channels":[1,6,7,11,14,17,24,28,33,40,44,46,50,53,56,60,63]},
    {"layer":'F6', "channels":[9,14,17,21,29,30,34,41,42,43,45,53,54,56,61,62]},
    
    {"layer":'G1', "channels":[9,10,15,23,34,36,39,42,44,62,72,73,76,85,90,91,94,107,109]},
    {"layer":'G2', "channels":[1,7,12,14,15,16,22,30,36,38,42,54,66,76,84,86,89,91,98,120,130,136,139,142]},
    {"layer":'G3', "channels":[11,23,33,45,52,67,90,95,105,123,145,185,191,199,232,248,283]},
    {"layer":'G4', "channels":[1,7]},
    {"layer":'G5', "channels":[3,14,21,56]},
    {"layer":'G6', "channels":[2,3,5,10,11,14,15,20,25,30,31,34,36,40,47,49,52,58]},
    
    {"layer":'H1', "channels":[1,5,11,22,34,45,54,57,55,66,93,99,102,107,116,125,155,158,174,183,219,225,250,255]},
    {"layer":'H2', "channels":[4,38,78,101]},
    {"layer":'H3', "channels":[5,7,8,18,42,43,59,77,79,103,136,144,149,192,200,230,233,268,270,282,303]},
    {"layer":'H4', "channels":[8,13]},
    {"layer":'H5', "channels":[9,20,26,34,37,43,67,89]},
    {"layer":'H6', "channels":[1,5,7,23,26,34,38,51,68,70,71,72,75,77,83,88,93,96,99,101,109,110,112,123,124]}
]

layers_m5 = [
    {"layer":'J1', "channels":[13,19,29,42,49,51,64,75,84,102,110,114,132,150,168,183,189,198,222,243,244]},
    {"layer":'J2', "channels":[9,18,32,48,69,159]},
    {"layer":'J3', "channels":[23,25,38,50,93,171,180,183,223,265,316]},
    {"layer":'J4', "channels":[8,21,42]},
    {"layer":'J5', "channels":[4,42,54,81,85,87,110,125]},
    {"layer":'J6', "channels":[16,17,53,54,65,71,75,82,89,103,109,120]},

    {"layer":'K1', "channels":[6,27,41,42,134,174,179,195,217,221,318,382]},
    {"layer":'K2', "channels":[1,7,9,22,33,52,87,101,109,113,146,155,189]},
    {"layer":'K3', "channels":[96,98,101,109,126,187,238,318,331]},
    {"layer":'K4', "channels":[2,12,41,46]},
    {"layer":'K5', "channels":[3,14,72,112]},
    {"layer":'K6', "channels":[17,41,80]}
]

layers_h1 = [
    {"layer":'L1', "channels":[50,97,46,125,18,126,112,9,114,35,20,28,10,60,119,124,48,108,67,11,86,87,110,59,103,127,69,91,44,21,56,41,55,96,64,89,92,83,17,]},
    {"layer":'L2', "channels":[65,96,125,20,81,15,84,31,1,54]}
]


faves_c1 = [
    {"layer":"A3", "channels":[89]}
]

faves_m3 = [
    {"layer":"B1", "channels":[5,31]},
    {"layer":"B2", "channels":[10,44]},
    {"layer":"B3", "channels":[27,62,76]},
    {"layer":"B4", "channels":[10]},
    {"layer":"B6", "channels":[15]},
    {"layer":"C1", "channels":[1,7,81,106]},
    {"layer":"C2", "channels":[0,44]},
    {"layer":"C3", "channels":[1,128,179]},
    {"layer":"C5", "channels":[39]},
    {"layer":"C6", "channels":[5,30,38,54]}
]

faves_m4 = [
    {"layer":"D1", "channels":[27,41,67,86,96,101,112,120,133,144,152,162,163,186,189]},
    {"layer":"D2", "channels":[13,50]},
    {"layer":"D3", "channels":[27,32,92,100]},
    {"layer":"D5", "channels":[15]},
    {"layer":"D6", "channels":[9,11,12,32]},
    {"layer":"E1", "channels":[100,115]},
    {"layer":"E2", "channels":[62]},
    {"layer":"F1", "channels":[40]},
    {"layer":"F2", "channels":[41]},
    {"layer":"F3", "channels":[147,187,254]},
    {"layer":"F5", "channels":[40,44,50,63]},
    {"layer":"F6", "channels":[9,17,34,41,42,54]},
    {"layer":"G1", "channels":[15,36,62,76,94]},
    {"layer":"G2", "channels":[1,7,15,30,36,42,54,91,136]},
    {"layer":"G6", "channels":[5,31,47,49,58]},
    {"layer":"H1", "channels":[5,22,34,102,107,116,158,183,225]},
    {"layer":"H2", "channels":[4,38]},
    {"layer":"H3", "channels":[7,77,103,136,303]},
    {"layer":"H4", "channels":[13]},
    {"layer":"H5", "channels":[34]},
    {"layer":"H6", "channels":[26,72,88,112,123]}
]

faves_m5 = [
    {"layer":"J1", "channels":[13,42,75,84,114]},
    {"layer":"J3", "channels":[25,223,265]},
    {"layer":"J5", "channels":[81]},
    {"layer":"J6", "channels":[54,103]},
    {"layer":"K1", "channels":[42,179]},
    {"layer":"K2", "channels":[1,113,146,126]},
    {"layer":"K5", "channels":[72]},
    {"layer":"K6", "channels":[41]}
]

faves_h1 = [
    {"layer":"L1", "channels":[9,18,20,28,35,41,46,55,86,97,103,108,124]}
]

birds=[{"layer":"F6","channels":[34]},{"layer":"G6","channels":[2,11,15]},{"layer":"H1","channels":[250]},{"layer":"H6","channels":[75,93]},{"layer":"J3","channels":[50]},{"layer":"J6","channels":[16]},{"layer":"L1","channels":[10]}]

bowl=[{"layer":"D1","channels":[96]},{"layer":"E1","channels":[62]},{"layer":"E2","channels":[78]},{"layer":"E3","channels":[150]},{"layer":"F2","channels":[32]},{"layer":"G2","channels":[38]},{"layer":"G3","channels":[185]},{"layer":"G6","channels":[14]},{"layer":"J3","channels":[171]},{"layer":"L1","channels":[56]}]

brass=[{"layer":"F6","channels":[56]},{"layer":"H1","channels":[1]},{"layer":"H6","channels":[124]},{"layer":"K2","channels":[7]},{"layer":"L1","channels":[91]}]

corner=[{"layer":"B2","channels":[44]},{"layer":"B6","channels":[15]},{"layer":"C6","channels":[54]},{"layer":"D1","channels":[41,101,120]},{"layer":"D3","channels":[92]},{"layer":"D5","channels":[15]},{"layer":"D6","channels":[11,12,52]},{"layer":"E6","channels":[21]},{"layer":"F1","channels":[78]},{"layer":"F1","channels":[94]}]

dof=[{"layer":"F2","channels":[7]},{"layer":"H2","channels":[38]},{"layer":"L1","channels":[48]}]

dogs=[{"layer":"G2","channels":[1]},{"layer":"G3","channels":[232]}]

eyes=[{"layer":"F3","channels":[48]},{"layer":"F5","channels":[7,24]},{"layer":"G2","channels":[1]},{"layer":"H1","channels":[250]}]

facade=[{"layer":"D1","channels":[130]},{"layer":"D2","channels":[6]},{"layer":"E3","channels":[95,130]},{"layer":"E6","channels":[33,44]},{"layer":"F2","channels":[6,64]},{"layer":"F3","channels":[75,95,107]},{"layer":"F6","channels":[29,61]},{"layer":"G2","channels":[66]},{"layer":"G6","channels":[5,10,25,49]},{"layer":"H1","channels":[158]},{"layer":"H3","channels":[77]},{"layer":"H6","channels":[5]},{"layer":"J1","channels":[49]},{"layer":"L1","channels":[59]}]

flower=[{"layer":"E3","channels":[17,36]},{"layer":"E6","channels":[16]},{"layer":"F2","channels":[120]},{"layer":"F3","channels":[230]},{"layer":"F6","channels":[17]},{"layer":"G2","channels":[139]},{"layer":"G3","channels":[145]},{"layer":"G5","channels":[21]},{"layer":"H3","channels":[43,136,192]}]

food=[{"layer":"E3","channels":[188]},{"layer":"F1","channels":[30]},{"layer":"F3","channels":[26]},{"layer":"F3","channels":[109]},{"layer":"G1","channels":[76]},{"layer":"G2","channels":[76,142]},{"layer":"G3","channels":[52,90,283]},{"layer":"G6","channels":[52]},{"layer":"H1","channels":[54]},{"layer":"H6","channels":[7]},{"layer":"J1","channels":[183,244]},{"layer":"L1","channels":[44,55]},{"layer":"L2","channels":[15]}]

nature=[{"layer":"D2","channels":[92]},{"layer":"D3","channels":[118,147]},{"layer":"D6","channels":[31]},{"layer":"E1","channels":[58]},{"layer":"E6","channels":[17]},{"layer":"F1","channels":[40]},{"layer":"F4","channels":[20]},{"layer":"F6","channels":[14]},{"layer":"G2","channels":[42]},{"layer":"H1","channels":[116]},{"layer":"H2","channels":[4]},{"layer":"H3","channels":[8,233,282]},{"layer":"H5","channels":[9]},{"layer":"H6","channels":[34,51]},{"layer":"J1","channels":[51,198]},{"layer":"J5","channels":[42]},{"layer":"J6","channels":[65]},{"layer":"J6","channels":[103]}]

pattern=[{"layer":"C6","channels":[38]},{"layer":"D6","channels":[32]},{"layer":"E6","channels":[63]},{"layer":"F5","channels":[33]},{"layer":"F5","channels":[56]},{"layer":"G2","channels":[86]}]

pipe=[{"layer":"E6","channels":[0]},{"layer":"F6","channels":[54]},{"layer":"G1","channels":[34,85]},{"layer":"L1","channels":[89,92]}]

sea=[{"layer":"E3","channels":[15]},{"layer":"F6","channels":[53]},{"layer":"F6","channels":[62]},{"layer":"G1","channels":[9,36]},{"layer":"G2","channels":[16,89]},{"layer":"G3","channels":[45,105]},{"layer":"H1","channels":[22,107]},{"layer":"H6","channels":[23,70,83]},{"layer":"J6","channels":[109]},{"layer":"L1","channels":[83]}]

simple=[{"layer":"A1","channels":[7,52]},{"layer":"A2","channels":[18,25]},{"layer":"A3","channels":[19,30,89,150,159,170]},{"layer":"B1","channels":[5,8]},{"layer":"B3","channels":[7,13,27]}]

stones=[{"layer":"G2","channels":[54]},{"layer":"H6","channels":[72]}]

thread=[{"layer":"B1","channels":[31]},{"layer":"B6","channels":[6]},{"layer":"C1","channels":[33]},{"layer":"C2","channels":[0,44]},{"layer":"D1","channels":[67]},{"layer":"D2","channels":[55]},{"layer":"D3","channels":[174,185,201]},{"layer":"E1","channels":[115,125]},{"layer":"E2","channels":[52]},{"layer":"E6","channels":[38]},{"layer":"F3","channels":[254]},{"layer":"G3","channels":[248]},{"layer":"H1","channels":[66]},{"layer":"H6","channels":[77,101,123]},{"layer":"J2","channels":[32]},{"layer":"L1","channels":[48,119,125,126]}]

wheel=[{"layer":"G6","channels":[20]},{"layer":"H5","channels":[20,89]}]






fave_moving1 = [
    {"layer":'A1', "channels":[15,52,7,8]},
    {"layer":'A2', "channels":[14,29]},
    {"layer":'A3', "channels":[124,130,150,170,19,30,31,58,75]},
    {"layer":'B1', "channels":[31,39,42]},
    {"layer":'B2', "channels":[10]},
    {"layer":'B3', "channels":[51,76]},
    {"layer":'C1', "channels":[7,81]},
    {"layer":'C2', "channels":[6]},
    {"layer":'C5', "channels":[39]},
    {"layer":'C6', "channels":[38,5,54]},
    {"layer":'D1', "channels":[120,41,63,96]},
    {"layer":'D2', "channels":[50]},
    {"layer":'D3', "channels":[130,185,27,92]},
    {"layer":'D5', "channels":[15]},
    {"layer":'D6', "channels":[27,31,32]},
    {"layer":'E2', "channels":[62]},
    {"layer":'E6', "channels":[27]},
    {"layer":'F1', "channels":[40]},
    {"layer":'F3', "channels":[187]},
    {"layer":'F5', "channels":[63]},
    {"layer":'F6', "channels":[9]},
    {"layer":'G2', "channels":[130,139,30,7,84]},
    {"layer":'G3', "channels":[232]},
    {"layer":'G6', "channels":[40]},
    {"layer":'H1', "channels":[107,22]},
    {"layer":'H2', "channels":[38]},
    {"layer":'H5', "channels":[37]},
    {"layer":'H6', "channels":[88]},
    {"layer":'J1', "channels":[102]}
]

fave_moving2 = [
    {"layer":'D3', "channels":[27,85]},
    {"layer":'D6', "channels":[9,52]},
    {"layer":'E1', "channels":[125]},
    {"layer":'E3', "channels":[17 ]},
    {"layer":'F1', "channels":[31,49,78]},
    {"layer":'F2', "channels":[64]},
    {"layer":'F3', "channels":[245]},
    {"layer":'G2', "channels":[1,16]},
    {"layer":'G3', "channels":[283]}
]


fave_moving = [
    {"layer":'A1', "channels":[15,52,7,8]},
    {"layer":'A2', "channels":[14,29]},
    {"layer":'A3', "channels":[124,130,150,170,19,30,31,58,75]},
    {"layer":'B1', "channels":[31,39,42,5]},
    {"layer":'B2', "channels":[10]},
    {"layer":'B3', "channels":[51,76]},
    {"layer":'C1', "channels":[7,81,77]},
    {"layer":'C2', "channels":[6,44]},
    {"layer":'C3', "channels":[179]},
    {"layer":'C4', "channels":[10]},
    {"layer":'C5', "channels":[39]},
    {"layer":'C6', "channels":[30,38,5,54]},
    {"layer":'D1', "channels":[120,144,41,63,96]},
    {"layer":'D2', "channels":[50]},
    {"layer":'D3', "channels":[85,100,33,130,185,27,92]},
    {"layer":'D5', "channels":[15]},
    {"layer":'D6', "channels":[9,52,27,31,32]},
    {"layer":'E1', "channels":[125,115,58]},
    {"layer":'E2', "channels":[62]},
    {"layer":'E3', "channels":[17]},
    {"layer":'E4', "channels":[15]},
    {"layer":'E6', "channels":[27,0]},
    {"layer":'F1', "channels":[40,31,49,78]},
    {"layer":'F2', "channels":[64,41]},
    {"layer":'F3', "channels":[187,245,254]},
    {"layer":'F5', "channels":[63,63]},
    {"layer":'F6', "channels":[9,56,41]},
    {"layer":'G1', "channels":[62,34]},
    {"layer":'G2', "channels":[1,16,130,139,30,7,84]},
    {"layer":'G3', "channels":[283,232,52]},
    {"layer":'G6', "channels":[40]},
    {"layer":'H1', "channels":[107,22,225]},
    {"layer":'H2', "channels":[38]},
    {"layer":'H3', "channels":[77,79]},
    {"layer":'H5', "channels":[37]},
    {"layer":'H6', "channels":[88]},
    {"layer":'J1', "channels":[102,42]},
    {"layer":'J2', "channels":[48]},
    {"layer":'J6', "channels":[65]}
]



fnt = ImageFont.truetype('/Users/gene/Code/of_v0.9.6_osx_release/apps/myApps/audioAnalyser-mfccInput_Classification/bin/data/Arial.ttf', 30)

def make_swatches(layer, img_name, s):
    swatches = []
    for fave in layer:
        name = fave['layer']
        layer_name = layer_lookup[name]
        for c in fave['channels']:
            swatch = {'path':'/Users/gene/bin/lapnorm/swatches/all/%s/c%04d.png'%(layer_name,c),
                      'name':'%s%d'%(name,c)}
            swatches.append(swatch)
    w = math.ceil(math.sqrt(len(swatches)))
    h = math.ceil(len(swatches)/w)
    im_full = Image.new("RGB", [int(w*s), int(h*s)])
    ctx = ImageDraw.Draw(im_full)
    for i,swatch in enumerate(swatches):
        x = i % w
        y = math.floor(float(i)/w)
        im = Image.open(swatch['path'])
        if im.width != s or im.height != s:
            im = im.resize((s,s))
        im_full.paste(im, (int(x*s), int(y*s)))
        ctx.text((int(x*s+1), int(y*s+1)), swatch['name'], font=fnt, fill=(255,255,255,220))
    im_full.save('/Users/gene/bin/lapnorm/swatches/faves_swatches/full_%s.png'%img_name)



# make_swatches(layers_c1, 'layers_c1', 305)
# make_swatches(layers_m3, 'layers_m3', 305)
# make_swatches(layers_m4, 'layers_m4', 305)
# make_swatches(layers_m5, 'layers_m5', 305)
# make_swatches(layers_h1, 'layers_h1', 305)
# make_swatches(faves_c1, 'faves_c1', 305)
# make_swatches(faves_m3, 'faves_m3', 305)
# make_swatches(faves_m4, 'faves_m4', 305)
# make_swatches(faves_m5, 'faves_m5', 305)
# make_swatches(faves_h1, 'faves_h1', 305)
# make_swatches(birds, 'birds', 305)
# make_swatches(bowl, 'bowl', 305)
# make_swatches(brass, 'brass', 305)
# make_swatches(corner, 'corner', 305)
# make_swatches(dof, 'dof', 305)
# make_swatches(dogs, 'dogs', 305)
# make_swatches(eyes, 'eyes', 305)
# make_swatches(facade, 'facade', 305)
# make_swatches(flower, 'flower', 305)
# make_swatches(food, 'food', 305)
# make_swatches(nature, 'nature', 305)
# make_swatches(pattern, 'pattern', 305)
# make_swatches(pipe, 'pipe', 305)
# make_swatches(sea, 'sea', 305)
# make_swatches(simple, 'simple', 305)
# make_swatches(stones, 'stones', 305)
# make_swatches(thread, 'thread', 305)
# make_swatches(wheel, 'wheel', 305)
# make_swatches(fave_moving1, 'fave_moving1', 305)
# make_swatches(fave_moving2, 'fave_moving2', 305)

make_swatches(fave_moving, 'fave_moving3', 305)


106
(0.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0013.png')
(1.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0019.png')
(2.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0029.png')
(3.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0042.png')
(4.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0049.png')
(5.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0051.png')
(6.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0064.png')
(7.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0075.png')
(8.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0084.png')
(9.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0102.png')
(10.0, 0.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0110.png')
(0.0, 1.0, '/Users/gene/bin/lapnorm/swatches/all/mixed5a_1x1_pre_relu/c0114.png')
(1.0, 1.0, 

In [1]:
## %matplotlib inline
import IPython.display
from PIL import Image, ImageStat, ImageEnhance
import math
import random
import os
from os import listdir
from os.path import isfile, isdir, join
import numpy as np
import scipy.misc
from tqdm import tqdm


print(os.isfile('yo'))

AttributeError: module 'os' has no attribute 'isfile'